In [2]:
import torch
from torch import nn
from torch.nn import functional as F

# 实例化nn.Sequential来构建我们的模型
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20) # 生成2*20的张量
net(X)

tensor([[-0.1732, -0.0013,  0.0949, -0.1733,  0.2603, -0.0388, -0.1496, -0.2707,
          0.2335, -0.0467],
        [-0.1703,  0.1043,  0.1591, -0.1438,  0.1208,  0.0669, -0.0835, -0.3359,
          0.1329, -0.0353]], grad_fn=<AddmmBackward>)

## 自定义块实现上面nn.Sequential的功能

In [3]:
class MLP(nn.Module):
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        super().__init__()
        self.hidden = nn.Linear(20, 256) # 隐藏层，使用nn.Linear自动帮我们生成了所需的权重矩阵、偏置,且有默认的参数初始化
        self.out = nn.Linear(256, 10) # 输出层
    
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

net = MLP()
net(X)

tensor([[ 0.0340,  0.0295,  0.3688, -0.1773, -0.0219, -0.4991, -0.1923, -0.0603,
         -0.3021, -0.1542],
        [ 0.1218,  0.0730,  0.2997, -0.2584, -0.0041, -0.4738, -0.1215, -0.1146,
         -0.2262, -0.1472]], grad_fn=<AddmmBackward>)

## 顺序块nn.Sequential的实现原理

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        """*args:继承nn.Nodule的子类实例"""
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在 Module类的成员变量_modules中。
            # _module的类型是OrderedDict（有序的字典）
            # _modules的主要优点是： 在模块的参数初始化过程中，系统知道在_modules字典中查找需要初始化参数的子块。
            self._modules[str(idx)] = module
    
    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X
    
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1791,  0.0017,  0.2754, -0.0058, -0.0905,  0.2018,  0.1406, -0.1256,
         -0.1665,  0.2240],
        [-0.0798, -0.0077,  0.1465,  0.0199, -0.0561,  0.2853,  0.0881, -0.1341,
         -0.1434,  0.1432]], grad_fn=<AddmmBackward>)

## 不是所有架构都是简单的顺序架构，自定义块拥有更强的灵活性

In [5]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数，其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False) 
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数和固定值的bias=1来模拟了一个全连接层的功能，随后通过relu
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数（同一个nn.Linear的对象，参数肯定一样）
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum() # 标量

net = FixedHiddenMLP()
net(X)

tensor(0.3314, grad_fn=<SumBackward0>)